In [0]:

df1 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-10").cache()
df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-05-10").cache()

df1.filter("est=10290 ").show()
df2.filter("app_id=1426252715 and free_app_download=10290").show()

In [0]:

nums = range(0, 10)
print(nums)
rdd = sc.parallelize(nums, 10)
rdd.map(lambda (x, y) : (x, x*2) ).collect()
print("Default parallelism: {}".format(sc.defaultParallelism))
print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))


In [0]:

test_data = [(x, y*y) for x in xrange(1,1000) for y in xrange(1,100)]
# print test_data

In [0]:

rdd = sc.parallelize(test_data, 3)
result1 = rdd.reduceByKey(lambda a, b: a + b).collect()
result2 = rdd.groupByKey().collect()
# print result

In [0]:

print result1[5]
print result2[5]

In [0]:


print dir(df2)

In [0]:

from pyspark.sql.functions import broadcast

df1.unpersist()
df2.unpersist()
df1 = spark.read.option("basePath",
                      "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-{01,03}/device_code=ios-tablet").cache()
# df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-01/device_code=ios-tablet").cache()
df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-01-01/device_code=ios-tablet/").cache()

df1.join(broadcast(df2), df1.app_id == df2.app_id).count()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-01/device_code=ios-tablet/ --human
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-01-01/device_code=ios-tablet/  --human
 


In [0]:

# sc.parallelize([("Alice","10","135215"),("Lisa","12","12456")])
lines = sc.parallelize(["hello world", "hi"])
words = lines.flatMap(lambda line: line.split(" ")) 
print words
print words.first()
# sc.parallelize
df1 = spark.createDataFrame([("12","12456","Lisa"),("10","135215","Alice")], schema=["age","tel","name"])
df2 = spark.createDataFrame([("Alice","10","135215"),("Lisa","12","12456"),("Lisa","12","12456")], schema=["name","age","tel"])

df1.printSchema()
df1.createOrReplaceTempView("view1")
df2.createOrReplaceTempView("view2")

spark.sql("select * from view1 except select * from view2").show()
spark.sql("select * from view2 except select * from view1").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/result_store_est_unified_db_count/

In [0]:
%%sh
aws s3 rm s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_monthly_category_count/ --recursive

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

start = "2018-02-11"
end = "2018-02-18"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()

for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
print test_path[0][2]


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""

"""
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2018-02-15"]
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select distinct date from raw_data where country_code='US'").show()